In [1]:
import os, sys
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
#!cd  ---> displaying current directory in windows
#!pwd ---> displaying current directory in linux

In [2]:
from codebase.data import datamodule

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


In [3]:
data1 = datamodule(None)

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:122: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:141: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [4]:
data1.prepare_data()

In [5]:
data1.setup()

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:115: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:134: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [6]:
print(data1)

Image Dataset
Num classes: 2
Mapping: {0: 'Ants', 1: 'Bees'}
Dims: (3, 224, 224)
Train/val/test sizes: 244, 153, 0
Batch x stats: (torch.Size([4, 3, 224, 224]), torch.float32, tensor(-2.1179), tensor(-0.1728), tensor(1.2115), tensor(2.6400))
Batch y stats: (torch.Size([4]), torch.int64, tensor(0), tensor(1))



In [8]:
# example demonstrating use of args from notebook
import argparse
parser = argparse.ArgumentParser()
datamodule.add_to_argparse(parser)
args = parser.parse_args(['--batch_size','16'])
data2 = datamodule(args)

In [9]:
data2.prepare_data()
data2.setup()
print(data2)

Image Dataset
Num classes: 2
Mapping: {0: 'Ants', 1: 'Bees'}
Dims: (3, 224, 224)
Train/val/test sizes: 244, 153, 0
Batch x stats: (torch.Size([16, 3, 224, 224]), torch.float32, tensor(-2.1179), tensor(0.0721), tensor(1.1943), tensor(2.6400))
Batch y stats: (torch.Size([16]), torch.int64, tensor(0), tensor(1))



In [7]:
x,y = next(iter(data1.train_dataloader()))
print(x.shape)
print(y.shape)

torch.Size([4, 3, 224, 224])
torch.Size([4])


In [8]:
data_config = data1.config()
data_config

{'input_dims': (3, 224, 224),
 'output_dims': (1,),
 'mapping': {0: 'Ants', 1: 'Bees'}}

In [9]:
from codebase.model.resnet import resnet18

model = resnet18(data_config)

output = model(x)

print(output.shape)

torch.Size([4, 2])


In [10]:
import pytorch_lightning as pl
from codebase.lit_model import BaseLitModel

In [11]:
logger = pl.loggers.TensorBoardLogger(BaseLitModel.log_dirname())
experiment_dir = logger.log_dir

In [12]:
goldstar_metric = "validation/loss"
filename_format = "epoch={epoch:04d}-validation.loss={validation/loss:.3f}"
checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=5,
        filename=filename_format,
        monitor=goldstar_metric,
        mode="min",
        auto_insert_metric_name=False,
        dirpath=experiment_dir,
        every_n_epochs= 1, # can be passed as an argument
    )

In [13]:
# testing 
batch = next(iter(data1.train_dataloader()))
lit_mode = BaseLitModel(model)
outputs = lit_mode.training_step(batch,0)
print(outputs)

{'loss': tensor(0.6423, grad_fn=<NllLossBackward0>)}


g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\lightning.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."


In [14]:
lit_mode = BaseLitModel(model)

summary_callback = pl.callbacks.ModelSummary(max_depth=2)

callbacks = [summary_callback, checkpoint_callback]
    


trainer = pl.Trainer( callbacks=callbacks, logger=logger,max_epochs=5)

    

trainer.fit(lit_mode, datamodule=data1)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "

  | Name        | Type     | Params
-------

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 4: 100%|██████████| 100/100 [00:25<00:00,  3.98it/s, loss=0.42, v_num=0, validation/loss=0.273, validation/acc=0.895] 


In [36]:
#path = logger.log_dir
#print(path)
#path = "/home/aravind/Coding Environment/Image Classification-1/log/lightning_logs/"
#%load_ext tensorboard
#%reload_ext tensorboard
#%tensorboard --logdir path

In [30]:
path = "E:\Coding Environment\Image Classification-1"
os.chdir(path)

In [31]:
!cd

E:\Coding Environment\Image Classification-1


In [35]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs/

Reusing TensorBoard on port 6006 (pid 6536), started 0:00:11 ago. (Use '!kill 6536' to kill it.)